# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-01-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass


# playground
print(file_path_list[0])
raise StopExecution

print('Hello world!')

/home/workspace/event_data/2018-11-13-events.csv


In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)
# raise StopExecution
# print('Hello world!')

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # original event csv columns: artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# compare len(full_data_rows_list) vs event_datafile_new
# 8056 vs 6821 because artist column is empty in some rows in original event csv!

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) # if you have a locally installed Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [21]:
# Create 1st table song_info_by_session
query = "DROP TABLE IF EXISTS song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# partition key = session_id (for partition)
# clustering column = item_in_session (for alphabeta ordering)
# columns = session_id, item_in_session, artist, song, length
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [22]:
# Insert 1st table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
#         print(query, (line[8], line[3], line[0], line[9], line[5]))

# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
* given the populated song_info_by_session, we now query to answer the 1st question. 
* We'll use session_id=338 the partition key to locate rows, and alphabeta ordering the results with item_in_session clustering column.
* `SELECT artist, song, length from song_info_by_session where session_id=338 and item_in_session=4`

In [23]:
query = 'SELECT artist, song, length from song_info_by_session where session_id=338 and item_in_session=4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [24]:
# Create 2nd TABLE artist_song_user_by_userid_sessionid
query = "DROP TABLE IF EXISTS artist_song_user_by_userid_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# partition key = user_id, session_id (for partition, and to be unique)
# clustering columns = item_in_session (for alphabeta ordering)
# columns = user_id, session_id, item_in_session, artist, first_name, last_name
query = "CREATE TABLE IF NOT EXISTS artist_song_user_by_userid_sessionid "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try: 
    session.execute(query)
except Exception as e:
    print(e)
                    

In [25]:
# Insert 2nd table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user_by_userid_sessionid (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
* given the populated artist_song_user_by_userid_sessionid, we now query to answer the 2nd question. 
* We use user_id=10 and session_id=182 as the composite partition key to locate rows
* SQL: `SELECT artist, song, first_name, last_name from artist_song_user_by_userid_sessionid where user_id=10 and session_id=182`

In [26]:
query = "SELECT artist, song, first_name, last_name from artist_song_user_by_userid_sessionid where user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [27]:
# CREATE 3rd TABLE user_info_by_song
query = "DROP TABLE IF EXISTS user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# partition key = song (for partition)
# clustering columns = user_id (for alphabeta ordering)
# columns = song, user_id, first_name, last_name
query = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [28]:
# Insert 3rd table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
* given the populated user_info_by_song, we now ready to answer the 3rd question
* We use song as the partition key to located rows, and no need to pass clustering column user_id.
* The reason we add user_id into primary key is to make primary key uniquely identify row in table, because a song maybe listened by multiple user, and it'll be overwritten in user_info_by_song table if primary key is only song.

In [18]:
query = "SELECT first_name, last_name from user_info_by_song where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [19]:
##  Drop the table before closing out the sessions

In [20]:
query = "DROP TABLE song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_song_user_by_userid_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table user_info_by_song"


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()